In [55]:
from hana_ml import dataframe
url, port, user = "hana-ml-api.hana-ml.c.ap-cn-1.cloud.sap", 30015, "PAL_TEST"
connection_context = dataframe.ConnectionContext(url, port, user)

In [56]:
from gen_ai_hub.proxy.core.proxy_clients import set_proxy_version
from gen_ai_hub.proxy.langchain import init_llm
#use btp proxy llm

llm = init_llm('gpt-4', temperature=0.0, max_tokens=512)

In [ ]:

from hana_ai.agents.hana_sql_agent import create_hana_sql_agent
from hana_ai.tools.toolkit import HANAMLToolkit
from hana_ai.vectorstore.code_templates import get_code_templates
from hana_ai.vectorstore.embedding_service import GenAIHubEmbeddings
from hana_ai.vectorstore.hana_vector_engine import HANAMLinVectorEngine

toolkit = HANAMLToolkit(connection_context)
embedding_func = GenAIHubEmbeddings()
hana_vec = HANAMLinVectorEngine(connection_context, "hana_vec_hana_ml_sql_knowledge")
hana_vec.create_knowledge(option='sql')
toolkit.set_vectordb(hana_vec)
agent_executor = create_hana_sql_agent(llm, connection_context, toolkit=toolkit,verbose=True)

100%|██████████| 2/2 [00:10<00:00,  5.40s/it]
c:\Users\I308290\AppData\Local\miniconda3\Lib\site-packages\sqlalchemy_hana\dialect.py:889: SAWarning: Did not recognize type 'REAL_VECTOR' of column 'embeddings'
  util.warn(


In [58]:
connection_context.table("hana_vec_hana_ml_sql_knowledge").head(5).collect()

96


,id,description,example,embeddings
0,tutorial-dataset-f9e32f5,The tutorial dataset is a classification datas...,\n\nSET SCHEMA DM_PAL;\n\nDROP TABLE PAL_AUTOM...,"[0, 3, 0, 0, 202, 255, 143, 188, 143, 8, 212, ..."
1,unified-classification-8bd88cf,The unified classification interface is a new ...,\n\nSET SCHEMA DM_PAL;\n\nDROP TABLE PAL_DATA...,"[0, 3, 0, 0, 141, 199, 72, 189, 113, 157, 219,..."
2,unified-classification-with-pivoted-input-data...,The content discusses a new unified interface ...,\n\nSET SCHEMA DM_PAL;\n\nDROP TABLE PAL_META_...,"[0, 3, 0, 0, 219, 37, 228, 188, 102, 207, 7, 1..."
3,unified-clustering-6870a0c,Unified clustering is a tool that provides a u...,\n\nSET SCHEMA DM_PAL;\n\nDROP TABLE MY_DATA_T...,"[0, 3, 0, 0, 128, 220, 70, 189, 107, 2, 116, 1..."
4,unified-clustering-with-pivoted-input-data-baa...,The content discusses unified clustering with ...,\n\nSET SCHEMA DM_PAL;\n\nDROP TABLE META_DATA...,"[0, 3, 0, 0, 196, 11, 94, 189, 44, 173, 244, 1..."


In [59]:
agent_executor.invoke(
    "Show me all the table names"
)



> Entering new SQL Agent Executor chain...
I need to list all the tables in the database.
Action: sql_db_list_tables
Action Input: ""boston_housing_prices, hana_vec_hana_ml_knowledge, hana_vec_hana_ml_sql_knowledge, iris_data_full_tbl, titanic_full_tblI now know the table names in the database.
Final Answer: The table names in the database are: boston_housing_prices, hana_vec_hana_ml_knowledge, hana_vec_hana_ml_sql_knowledge, iris_data_full_tbl, titanic_full_tbl.

> Finished chain.


{'input': 'Show me all the table names',
 'output': 'The table names in the database are: boston_housing_prices, hana_vec_hana_ml_knowledge, hana_vec_hana_ml_sql_knowledge, iris_data_full_tbl, titanic_full_tbl.'}

In [60]:
agent_executor.invoke(
    "show me the column names in the table related to iris"
)



> Entering new SQL Agent Executor chain...
First, I need to confirm if there is a table related to iris in the database.
Action: sql_db_list_tables
Action Input: ""boston_housing_prices, hana_vec_hana_ml_knowledge, hana_vec_hana_ml_sql_knowledge, iris_data_full_tbl, titanic_full_tblThere is a table named 'iris_data_full_tbl' which seems to be related to iris. Now, I need to get the schema of this table to see the column names.
Action: sql_db_schema
Action Input: "iris_data_full_tbl"
CREATE TABLE iris_data_full_tbl (
	id INTEGER, 
	sepallengthcm DOUBLE, 
	sepalwidthcm DOUBLE, 
	petallengthcm DOUBLE, 
	petalwidthcm DOUBLE, 
	species NVARCHAR(5000)
)

/*
3 rows from iris_data_full_tbl table:
id	sepallengthcm	sepalwidthcm	petallengthcm	petalwidthcm	species
0	5.1	3.5	1.4	0.2	Iris-setosa
1	4.9	3.0	1.4	0.2	Iris-setosa
2	4.7	3.2	1.3	0.2	Iris-setosa
*/I now know the column names in the table related to iris.
Final Answer: The column names in the table related to iris are: id, sepallengthcm, s

{'input': 'show me the column names in the table related to iris',
 'output': 'The column names in the table related to iris are: id, sepallengthcm, sepalwidthcm, petallengthcm, petalwidthcm, species.'}

In [61]:
agent_executor.invoke(
    "show me the min and max value of sepalwidthcm in the table iris"
)



> Entering new SQL Agent Executor chain...
I need to write a SQL query to get the min and max value of sepalwidthcm from the table iris.
Action: sql_db_query_checker
Action Input: SELECT MIN(sepalwidthcm), MAX(sepalwidthcm) FROM irisSELECT MIN(sepalwidthcm), MAX(sepalwidthcm) FROM irisThe query seems to be correct. Now I will execute it.
Action: sql_db_query
Action Input: SELECT MIN(sepalwidthcm), MAX(sepalwidthcm) FROM irisError: (hdbcli.dbapi.ProgrammingError) (259, 'invalid table name:  Could not find table/view IRIS in schema PAL_TEST: line 1 col 50 (at pos 49)')
[SQL: SELECT MIN(sepalwidthcm), MAX(sepalwidthcm) FROM iris]
(Background on this error at: https://sqlalche.me/e/20/f405)The error message indicates that the table 'iris' does not exist in the schema 'PAL_TEST'. I need to check the list of tables in the database.
Action: sql_db_list_tables
Action Input: ""boston_housing_prices, hana_vec_hana_ml_knowledge, hana_vec_hana_ml_sql_knowledge, iris_data_full_tbl, titanic_full_t

{'input': 'show me the min and max value of sepalwidthcm in the table iris',
 'output': 'The minimum value of sepalwidthcm in the table iris_data_full_tbl is 2.0 and the maximum value is 4.4.'}